# This notebook is designed for teaching/testing purposes to help you visualize the tensor shapes that go through each module

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

In [2]:
# config file
from config import ModelConfig
cfg = ModelConfig()
print(cfg)

# import the tokenizer specified by cfg
from tools import import_from_nested_path
imported_objects = import_from_nested_path(['tokenizers', cfg.tokenizer], 'tokenizer', ['get_tokenizer'])
get_tokenizer = imported_objects.get('get_tokenizer')
tokenizer = get_tokenizer(size = 512) # assuming 'bpe', size options are 95, 128, 256, 512, 1024 and 2048

import random
import torch

ModelConfig(dim=128, device='cpu', tokenizer='bpe_v1', vocab_len=8192, num_layers=6, second_resid_norm=False, mlp_hidden_mult=4, mlp_bias=False, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=2, num_kv_heads=1, head_dim=64, theta=10000, max_seq_len=64, ca_num_q_heads=2, ca_num_kv_heads=1, ca_head_dim=64, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, pool_type='sum', pre_pool_norm=True, pool_output_linear=False, pool_bias=False, compress_freq='linear', compress_freq_n=1, fs_mult_factor=4, fs_max_iter=3, max_batch_size=1)


# Norms

In [3]:
from modules.norm import Norm

In [4]:
%%time

### RMSNorm

# Create an instance of RMSNorm
module = Norm(cfg.dim, 'RMSNorm').to(cfg.device)

# let's take a look
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)

# Call the forward method - logging will occur
output = module(x)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, x, output

0.256 K parameters
Norm()

==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])

==========Entering Norm.RMSNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting Norm.RMSNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting Norm.forward==========
CPU times: user 5.61 ms, sys: 2.75 ms, total: 8.36 ms
Wall time: 5.8 ms


In [5]:
%%time

# LayerNorm
module = Norm(cfg.dim, 'LayerNorm').to(cfg.device)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)
output = module(x)
module.disable_logging()
del module, x, output


==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])

==========Entering Norm.LayerNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting Norm.LayerNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting Norm.forward==========
CPU times: user 10.9 ms, sys: 2.56 ms, total: 13.4 ms
Wall time: 7.07 ms


# Attention

In [6]:
from modules.mqa import MQA, precompute_freqs_cis

In [7]:
%%time

# first up let's look at training

# Create an instance of multi-head self-attention
module = MQA(
        cfg.dim,
        cfg.head_dim,
        cfg.num_q_heads,
        cfg.num_kv_heads,
        cfg.max_seq_len,
        #cfg.max_batch_size, # if you don't pass in a max_batch_size then the module will be incapable of kv caching
).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
).to(cfg.device)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)

# Call the forward method - logging will occur
output = module(x, x, x, freqs_cis, mask, training=True)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, x, output

49.152 K parameters
MQA(
  (Wq): Linear(in_features=128, out_features=128, bias=False)
  (Wk): Linear(in_features=128, out_features=64, bias=False)
  (Wv): Linear(in_features=128, out_features=64, bias=False)
  (Wo): Linear(in_features=128, out_features=128, bias=False)
)

==========Entering MQA.forward==========
Inputs:
Tensor 'q' shape: torch.Size([32, 64, 128])
Tensor 'k' shape: torch.Size([32, 64, 128])
Tensor 'v' shape: torch.Size([32, 64, 128])
Tensor 'freqs_cis' shape: torch.Size([64, 32])
Tensor 'mask' shape: torch.Size([64, 64])
Other-type 'cache_len': Type=NoneType, Value=None
Integer 'training': Value=True

==========Entering MQA.apply_rotary_emb==========
Inputs:
Tensor 'q' shape: torch.Size([32, 64, 2, 64])
Tensor 'k' shape: torch.Size([32, 64, 1, 64])
Tensor 'freqs_cis' shape: torch.Size([64, 32])

==========Entering MQA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([64, 32])
Tensor 'x' shape: torch.Size([32, 64, 2, 32])

Outputs:
Tensor 'ou

In [8]:
%%time

# now let's do it for inference

module = MQA(
        cfg.dim,
        cfg.head_dim,
        cfg.num_q_heads,
        cfg.num_kv_heads,
        cfg.max_seq_len,
        cfg.max_batch_size
).to(cfg.device)
module.enable_logging()
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
).to(cfg.device)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)

# setting up for kv caching
context_chunk_len = cfg.max_seq_len // 4
cache_len = random.randint(1, 3 * context_chunk_len)
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )

# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(cfg.max_batch_size,context_chunk_len,cfg.dim).to(cfg.device)

# Call the forward method - logging will occur
output = module(x, x, x, freqs_cis, mask, cache_len)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output


==========Entering MQA.forward==========
Inputs:
Tensor 'q' shape: torch.Size([1, 16, 128])
Tensor 'k' shape: torch.Size([1, 16, 128])
Tensor 'v' shape: torch.Size([1, 16, 128])
Tensor 'freqs_cis' shape: torch.Size([16, 32])
Tensor 'mask' shape: torch.Size([16, 20])
Integer 'cache_len': Value=4
Integer 'training': Value=False

==========Entering MQA.apply_rotary_emb==========
Inputs:
Tensor 'q' shape: torch.Size([1, 16, 2, 64])
Tensor 'k' shape: torch.Size([1, 16, 1, 64])
Tensor 'freqs_cis' shape: torch.Size([16, 32])

==========Entering MQA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([16, 32])
Tensor 'x' shape: torch.Size([1, 16, 2, 32])

Outputs:
Tensor 'output' shape: torch.Size([1, 16, 1, 32])
==========Exiting MQA.reshape_for_broadcast==========

Outputs:
Tensor 'output[0]' shape: torch.Size([1, 16, 2, 64])
Tensor 'output[1]' shape: torch.Size([1, 16, 1, 64])
==========Exiting MQA.apply_rotary_emb==========

==========Entering MQA.match_headcount=

In [9]:
%%time

# now cross-attention

# Create an instance of multi-head cross-attention
module = MQA(
        cfg.dim,
        cfg.head_dim,
        cfg.num_q_heads,
        cfg.num_kv_heads,
        cfg.max_seq_len,
        #cfg.max_batch_size, # if you don't pass in a max_batch_size then the module will be incapable of kv caching
).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)
z = torch.randn(32,1,cfg.dim).to(cfg.device)

# Call the forward method - logging will occur
output = module(x, z, z, training=True)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, x, output

49.152 K parameters
MQA(
  (Wq): Linear(in_features=128, out_features=128, bias=False)
  (Wk): Linear(in_features=128, out_features=64, bias=False)
  (Wv): Linear(in_features=128, out_features=64, bias=False)
  (Wo): Linear(in_features=128, out_features=128, bias=False)
)

==========Entering MQA.forward==========
Inputs:
Tensor 'q' shape: torch.Size([32, 64, 128])
Tensor 'k' shape: torch.Size([32, 1, 128])
Tensor 'v' shape: torch.Size([32, 1, 128])
Other-type 'freqs_cis': Type=NoneType, Value=None
Other-type 'mask': Type=NoneType, Value=None
Other-type 'cache_len': Type=NoneType, Value=None
Integer 'training': Value=True

==========Entering MQA.match_headcount==========
Inputs:
Tensor 'k' shape: torch.Size([32, 1, 1, 64])
Tensor 'v' shape: torch.Size([32, 1, 1, 64])

Outputs:
Tensor 'output[0]' shape: torch.Size([32, 1, 2, 64])
Tensor 'output[1]' shape: torch.Size([32, 1, 2, 64])
==========Exiting MQA.match_headcount==========

==========Entering MQA.attend==========
Inputs:
Tensor 'q'

# MLP

In [10]:
from modules.mlp import MLP

In [11]:
%%time

# GeGLU
module = MLP(
    cfg.dim, 
    int(cfg.dim * cfg.mlp_hidden_mult * 2/3), 
    cfg.dim, 
    'GeLU', 
    gated=True, 
    bias=False, 
    dropout_rate = 0.1
).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)
output = module(x, training=True)
module.disable_logging()
del module, x, output

130.944 K parameters
MLP(
  (Wup): Linear(in_features=128, out_features=341, bias=False)
  (Wgate): Linear(in_features=128, out_features=341, bias=False)
  (Wdown): Linear(in_features=341, out_features=128, bias=False)
  (nonlinearity): GELU(approximate='none')
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting MLP.forward==========
CPU times: user 14.2 ms, sys: 4.01 ms, total: 18.2 ms
Wall time: 12.7 ms


In [12]:
%%time

# not gated, testing every other nonlinearity
module = MLP(
    cfg.dim, 
    cfg.dim * cfg.mlp_hidden_mult, 
    cfg.dim, 
    'ReLU', 
    gated=False, 
    bias=False, 
    dropout_rate = 0.1
).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)
output = module(x, training=True)
module.disable_logging()
del module, x, output

131.072 K parameters
MLP(
  (Wup): Linear(in_features=128, out_features=512, bias=False)
  (Wdown): Linear(in_features=512, out_features=128, bias=False)
  (nonlinearity): ReLU()
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 64, 128])
==========Exiting MLP.forward==========
CPU times: user 13 ms, sys: 5.57 ms, total: 18.6 ms
Wall time: 13.4 ms


# ResidualLayer

In [13]:
from modules.layer import Layer

In [14]:
%%time

# TRAINING w/ only self-attention
module = Layer(cfg).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('self_attn_connect')
#module.disable_function_logging('mlp_connect')
### enabling printing for sub-modules
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
).to(cfg.device)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)

output = module(x, freqs_cis, mask, training=True)
module.disable_logging()
del module, freqs_cis, mask, x, output

180.608 K parameters
Layer(
  (pre_self_attn_norm): Norm()
  (self_attn): MQA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=64, bias=False)
    (Wv): Linear(in_features=128, out_features=64, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wup): Linear(in_features=128, out_features=341, bias=False)
    (Wgate): Linear(in_features=128, out_features=341, bias=False)
    (Wdown): Linear(in_features=341, out_features=128, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering Layer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 64, 128])
Tensor 'freqs_cis' shape: torch.Size([64, 32])
Tensor 'mask' shape: torch.Size([64, 64])
Other-type 'cache_len': Type=NoneType, Value=None
Other-type 'y': Type=NoneType, Value=None
Integer 'training': Value=True

==========Entering Layer.self_attn_connect==========
Inputs:
Tensor 'x' shape: torc

In [15]:
%%time

# INFERENCE
module = Layer(cfg).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('self_attn_connect')
#module.disable_function_logging('mlp_connect')
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
).to(cfg.device)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
# setting up for kv caching
cache_len = cfg.max_seq_len // 3
context_chunk_len = cfg.max_seq_len // 4
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )
# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(1,context_chunk_len,cfg.dim).to(cfg.device)

output = module(x, freqs_cis, mask, cache_len)
module.disable_logging()
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output

180.608 K parameters
Layer(
  (pre_self_attn_norm): Norm()
  (self_attn): MQA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=64, bias=False)
    (Wv): Linear(in_features=128, out_features=64, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wup): Linear(in_features=128, out_features=341, bias=False)
    (Wgate): Linear(in_features=128, out_features=341, bias=False)
    (Wdown): Linear(in_features=341, out_features=128, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering Layer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([1, 16, 128])
Tensor 'freqs_cis' shape: torch.Size([16, 32])
Tensor 'mask' shape: torch.Size([16, 37])
Integer 'cache_len': Value=21
Other-type 'y': Type=NoneType, Value=None
Integer 'training': Value=False

==========Entering Layer.self_attn_connect==========
Inputs:
Tensor 'x' shape: torch.Size([1, 16, 128])

In [16]:
%%time

# Layer w/ cross-attention and w/ kv caching in the self-attention enabled while TRAINING
module = Layer(cfg, cross_attn=True).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('self_attn_connect')
#module.disable_function_logging('cross_attn_connect')
#module.disable_function_logging('mlp_connect')
### enabling printing for sub-modules
#module.pre_self_attn_norm.enable_logging()
#module.self_attn.enable_logging()
#module.post_self_attn_norm.enable_logging()
#module.pre_cross_attn_norm_x.enable_logging()
#module.pre_cross_attn_norm_z.enable_logging()
#module.cross_attn.enable_logging()
#module.post_cross_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
).to(cfg.device)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim).to(cfg.device)
y = torch.randn(32, 1, cfg.dim).to(cfg.device)

output = module(x, freqs_cis, mask, y=y)
module.disable_logging()
del module, freqs_cis, mask, x, y, output

230.272 K parameters
Layer(
  (pre_self_attn_norm): Norm()
  (self_attn): MQA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=64, bias=False)
    (Wv): Linear(in_features=128, out_features=64, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_cross_attn_norm_x): Norm()
  (pre_cross_attn_norm_y): Norm()
  (cross_attn): MQA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=64, bias=False)
    (Wv): Linear(in_features=128, out_features=64, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wup): Linear(in_features=128, out_features=341, bias=False)
    (Wgate): Linear(in_features=128, out_features=341, bias=False)
    (Wdown): Linear(in_features=341, out_features=128, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering Layer.forward==========
I

In [21]:
%%time

# Layer w/ cross-attention and w/o kv caching in the self-attention enabled during INFERENCE
module = Layer(cfg, cross_attn = True, kv_cache = False).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('self_attn_connect')
#module.disable_function_logging('cross_attn_connect')
#module.disable_function_logging('mlp_connect')
#module.pre_self_attn_norm.enable_logging()
#module.self_attn.enable_logging()
#module.post_self_attn_norm.enable_logging()
#module.pre_cross_attn_norm_x.enable_logging()
#module.pre_cross_attn_norm_z.enable_logging()
#module.cross_attn.enable_logging()
#module.post_cross_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
).to(cfg.device)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(1,cfg.max_seq_len,cfg.dim).to(cfg.device)
y = torch.randn(1, 1, cfg.dim).to(cfg.device)

output = module(x, freqs_cis, mask, y=y)
module.disable_logging()
del module, freqs_cis, mask, x, y, output

230.272 K parameters
Layer(
  (pre_self_attn_norm): Norm()
  (self_attn): MQA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=64, bias=False)
    (Wv): Linear(in_features=128, out_features=64, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_cross_attn_norm_x): Norm()
  (pre_cross_attn_norm_y): Norm()
  (cross_attn): MQA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=64, bias=False)
    (Wv): Linear(in_features=128, out_features=64, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wup): Linear(in_features=128, out_features=341, bias=False)
    (Wgate): Linear(in_features=128, out_features=341, bias=False)
    (Wdown): Linear(in_features=341, out_features=128, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering Layer.forward==========
I

# Full Model

In [15]:
from modules.model import Model

In [16]:
%%time

# TRAINING
module = Model(cfg).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
module.layers[0].enable_logging()
module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len)).to(cfg.device)
target_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len)).to(cfg.device)

output, loss = module(input_token_ids, target_token_ids=target_token_ids)
print(loss)
module.disable_logging()
del module, input_token_ids, target_token_ids, output

4011.456 K parameters
Model(
  (token_embedder): Embedding(8195, 192)
  (layers): ModuleList(
    (0-5): 6 x Layer(
      (pre_attn_norm): Norm()
      (attn): MQA(
        (Wq): Linear(in_features=192, out_features=192, bias=False)
        (Wk): Linear(in_features=192, out_features=96, bias=False)
        (Wv): Linear(in_features=192, out_features=96, bias=False)
        (Wo): Linear(in_features=192, out_features=192, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wup): Linear(in_features=192, out_features=512, bias=False)
        (Wgate): Linear(in_features=192, out_features=512, bias=False)
        (Wdown): Linear(in_features=512, out_features=192, bias=False)
        (nonlinearity): SiLU()
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering Model.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 512])
Tensor 'cache_len' shape: torch.Size([32, 512])

==========Entering Layer.forward====

In [17]:
%%time

# Inference
module = Model(cfg).to(cfg.device)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
#for i in range(cfg.num_layers):
    #module.layers[i].enable_logging()
#module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (1, cfg.max_seq_len // 4)).to(cfg.device)

output, _ = module(input_token_ids, cache_len = cfg.max_seq_len // 3)
print(output.shape)
module.disable_logging()
del module, input_token_ids, output

4011.456 K parameters
Model(
  (token_embedder): Embedding(8195, 192)
  (layers): ModuleList(
    (0-5): 6 x Layer(
      (pre_attn_norm): Norm()
      (attn): MQA(
        (Wq): Linear(in_features=192, out_features=192, bias=False)
        (Wk): Linear(in_features=192, out_features=96, bias=False)
        (Wv): Linear(in_features=192, out_features=96, bias=False)
        (Wo): Linear(in_features=192, out_features=192, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wup): Linear(in_features=192, out_features=512, bias=False)
        (Wgate): Linear(in_features=192, out_features=512, bias=False)
        (Wdown): Linear(in_features=512, out_features=192, bias=False)
        (nonlinearity): SiLU()
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering Model.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([1, 128])
Integer 'cache_len': Value=170

Outputs:
Tensor 'output[0]' shape: torch.Size([1, 128,